# @author - Pardeep
# transfer learning model using vgg16
# importing transfer learning model

In [1]:
# import necessary library
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import Dense
# load and creare a model object
vggmodel = VGG16()
# create model summary
vggmodel.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

# preprocessing pre trained model

In [2]:
# checking type of the model wheater it is sequential or functional
type(vggmodel)

tensorflow.python.keras.engine.functional.Functional

In [3]:
# convert functional model to sequential
vggmodelse = Sequential()

In [4]:
# removing last layer for modification to use in our program
for layer in vggmodel.layers[:-1]:
    vggmodelse.add(layer)

In [5]:
# again checking the model type
type(vggmodelse)

tensorflow.python.keras.engine.sequential.Sequential

In [6]:
# checking summary of newly created sequential model
vggmodelse.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [7]:
# setting pretrained layers as they should not be trained again
for layer in vggmodelse.layers:
    layer.trainable=False

# adding last layer for classification

In [8]:
# adding last layer for catogorical sepration
vggmodelse.add(Dense(units=30, activation='sigmoid'))

In [9]:
vggmodelse.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

# Data Preprocessing

In [10]:
# using dplifolders splitting the data into train and test
import splitfolders

In [11]:
input_dataset="Segmented Medicinal Leaf Images"
output_dataset="splitted data"
splitfolders.ratio(input_dataset,output_dataset,seed=42,ratio=(.8,.2)) # will split in train test and validation

Copying files: 1835 files [00:02, 811.62 files/s]


In [13]:
from keras.preprocessing.image import ImageDataGenerator

### Preprocessing the Training set

In [22]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('splitted data/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 1455 images belonging to 30 classes.


### Preprocessing the Test set

In [23]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('splitted data/val',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 380 images belonging to 30 classes.


# Training the CNN using transfer learning

### Compiling the CNN

In [24]:
vggmodelse.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [25]:
vggmodelse.fit(x = training_set, validation_data = test_set, epochs = 10)

Epoch 1/10
46/46 [==============================] - 146s 3s/step - loss: 0.2651 - accuracy: 0.0850 - val_loss: 0.1293 - val_accuracy: 0.1921
Epoch 2/10
46/46 [==============================] - 150s 3s/step - loss: 0.1231 - accuracy: 0.2639 - val_loss: 0.1038 - val_accuracy: 0.3947
Epoch 3/10
46/46 [==============================] - 151s 3s/step - loss: 0.1030 - accuracy: 0.4193 - val_loss: 0.0911 - val_accuracy: 0.5368
Epoch 4/10
46/46 [==============================] - 152s 3s/step - loss: 0.0925 - accuracy: 0.5127 - val_loss: 0.0823 - val_accuracy: 0.5579
Epoch 5/10
46/46 [==============================] - 151s 3s/step - loss: 0.0842 - accuracy: 0.5612 - val_loss: 0.0749 - val_accuracy: 0.6474
Epoch 6/10
46/46 [==============================] - 152s 3s/step - loss: 0.0734 - accuracy: 0.6542 - val_loss: 0.0681 - val_accuracy: 0.7053
Epoch 7/10
46/46 [==============================] - 152s 3s/step - loss: 0.0700 - accuracy: 0.6670 - val_loss: 0.0615 - val_accuracy: 0.7447
Epoch 8/10
46

## Making a single prediction

In [39]:
from tensorflow import keras
import tensorflow as tf

In [42]:
img = keras.preprocessing.image.load_img(
    "Segmented Medicinal Leaf Images/Citrus Limon (Lemon)/CL-S-013.jpg", target_size=(224,224,3)
)
img_array = keras.preprocessing.image.img_to_array(img)/255
img_array = tf.expand_dims(img_array, 0)  # Create batch axis

predictions = vggmodelse.predict(img_array)
result = predictions[0]

In [43]:
if result[0] == result.max():
  prediction = 'Alpinia Galanga (Rasna)'
elif result[1] == result.max():
    prediction = 'Amaranthus Viridis (Arive-Dantu)'
elif result[2] == result.max():
    prediction = 'Artocarpus Heterophyllus (Jackfruit)'
elif result[3] == result.max():
    prediction = 'Azadirachta Indica (Neem)'
elif result[4] == result.max():
    prediction = 'Basella Alba (Basale)'
elif result[5] == result.max():
    prediction = 'Brassica Juncea (Indian Mustard)'
elif result[6] == result.max():
    prediction = 'Carissa Carandas (Karanda)'
elif result[7] == result.max():
    prediction = 'Citrus Limon (Lemon)'
elif result[8] == result.max():
    prediction = 'Ficus Auriculata (Roxburgh fig)'
elif result[9] == result.max():
    prediction = 'Ficus Religiosa (Peepal Tree)'
elif result[10] == result.max():
    prediction = 'Hibiscus Rosa-sinensis'
elif result[11] == result.max():
    prediction = 'Jasminum (Jasmine)'
elif result[12] == result.max():
    prediction = 'Mangifera Indica (Mango)'
elif result[12] == result.max():
    prediction = 'Mentha (Mint)'
elif result[14] == result.max():
    prediction = 'Moringa Oleifera (Drumstick)'
elif result[15] == result.max():
    prediction = 'Muntingia Calabura (Jamaica Cherry-Gasagase)'
elif result[16] == result.max():
    prediction = 'Murraya Koenigii (Curry)'
elif result[17] == result.max():
    prediction = 'Nerium Oleander (Oleander)'
elif result[18] == result.max():
    prediction = 'Nyctanthes Arbor-tristis (Parijata)'
elif result[19] == result.max():
    prediction = 'Ocimum Tenuiflorum (Tulsi)'
elif result[20] == result.max():
    prediction = 'Piper Betle (Betel)'
elif result[21] == result.max():
    prediction = 'Plectranthus Amboinicus (Mexican Mint)'
elif result[22] == result.max():
    prediction = 'Pongamia Pinnata (Indian Beech)'
elif result[23] == result.max():
    prediction = 'Psidium Guajava (Guava)'
elif result[24] == result.max():
    prediction = 'Punica Granatum (Pomegranate)'
elif result[25] == result.max():
    prediction = 'Santalum Album (Sandalwood)'
elif result[26] == result.max():
    prediction = 'Syzygium Cumini (Jamun)'
elif result[27] == result.max():
    prediction = 'Syzygium Jambos (Rose Apple)'
elif result[28] == result.max():
    prediction = 'Tabernaemontana Divaricata (Crape Jasmine)'
else:
    prediction = 'Trigonella Foenum-graecum (Fenugreek)'

In [44]:
print(prediction)

Citrus Limon (Lemon)


## saving a model

In [56]:
import tensorflow as tf

from keras.models import load_model

vggmodelse.save('vggmodelse')

INFO:tensorflow:Assets written to: vggmodelse\assets
